<a href="https://colab.research.google.com/github/Prach1/CS69002_9A_18CS60R30/blob/master/DL_Assign9a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import pandas as pd
import numpy as np
import io
import re

In [1]:
from google.colab import files
uploaded = files.upload()

Saving Train_20K.csv to Train_20K.csv


In [4]:
df = pd.read_csv(io.StringIO(uploaded['Train_20K.csv'].decode('utf-8')), sep='\t')
print(df.head())

                                                text  label
0  John Waters has given us a genuinely enjoyable...      1
1  This first two seasons of this comedy series w...      1
2  What an unfortunate mess is "Shiner." I wanted...      0
3  I'm not entirely sure Rob Schmidt qualifies as...      1
4  i wasn't sure whether to laugh or cry. Porrett...      0


In [0]:
for chunk in df:
  print(chunk)

text
label


In [5]:
# for i, j in df.iterrows(): 
#     print(i)
#     print(j)

print(type(df['text']))
print(type(df['label']))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [8]:
text_reviews = df['text'].astype(str).tolist()
text_labels = df['label'].astype(int)
#print(text_labels)

text_reviews = [x.lower() for x in text_reviews]
print(type(text_reviews))
print(text_reviews[0])

<class 'list'>
john waters has given us a genuinely enjoyable film. this certainly isn't without its shocking waters-esque moments, but it is tamer than his older culty stuff, such as "pink flamingoes". "pecker" harkens back to john's early mainstream stage in that it reminds the viewer of the same kind of humor that was evident in "polyester". overall, a really fun comedy with some great moments!


In [0]:
import re

def preprocess(text):    
  text = re.sub(r'<br.*?>',' ',text)
  text = re.sub(r'[^\w\s]',' ',text) 
  text = re.sub(r'[0-9]+', ' ', text)
  
  return text
  

In [30]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
#nltk.download('stopwords')
#nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer

train_data = []
train_data_X = []
stop = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

for review in text_reviews:
  new_review = preprocess(review)
  train_data.append(new_review)
  
print(train_data[0])

for review in train_data:
  review = [lemmatizer.lemmatize(word) for word in review.split() if word not in stop and word != '' and len(lemmatizer.lemmatize(word)) > 1]
  train_data_X.append(review)
  
print(train_data_X[0])

john waters has given us a genuinely enjoyable film  this certainly isn t without its shocking waters esque moments  but it is tamer than his older culty stuff  such as  pink flamingoes    pecker  harkens back to john s early mainstream stage in that it reminds the viewer of the same kind of humor that was evident in  polyester   overall  a really fun comedy with some great moments 
['john', 'water', 'given', 'genuinely', 'enjoyable', 'film', 'certainly', 'without', 'shocking', 'water', 'esque', 'moment', 'tamer', 'older', 'culty', 'stuff', 'pink', 'flamingo', 'pecker', 'harkens', 'back', 'john', 'early', 'mainstream', 'stage', 'reminds', 'viewer', 'kind', 'humor', 'evident', 'polyester', 'overall', 'really', 'fun', 'comedy', 'great', 'moment']


In [0]:
def generate_word_ids(dataset):
  word_to_ix = {}
 
  for sent in dataset:
      for word in sent:
          if word not in word_to_ix:
              word_to_ix[word] = len(word_to_ix)
              
  return word_to_ix


In [0]:
word_to_ix = generate_word_ids(train_data_X)
#print(word_to_ix)
VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2

In [0]:
def make_bow_vector(sentence, word_to_ix):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_ix))
    for word in sentence.split():
        if word not in word_to_ix:
            raise ValueError('Word',word,' not present in the dictionary. Sorry!')
        else:
            vec[word_to_ix[word]]+=1
    return vec.view(1, -1)

def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])